In [1]:
from tqdm import tqdm
import math
import numpy as np
import pandas as pd
from xml.etree import ElementTree
tree = ElementTree.parse("RK2015.xml")
root = tree.getroot()




pd_DistrictName = []
pd_DistrictNumber = []
pd_DistrictQuota = []
pd_DistrictMandates = []
pd.set_option('max_rows',50)




pd_forename = []
pd_surname = [] 
pd_candidateRegNumber = [] 
pd_candidateID = []
pd_partyName = [] 
pd_partyCode = []
pd_K_list = []
pd_P_list = []
pd_E_list = []
pd_J1P_list = []
pd_J1E_list = []
pd_J2P_list = []
pd_J2E_list = []
pd_J3P_list = [] 
pd_J3E_list = []
vote_vector = [] 
pd_adminUnit = []
pd_ehak_Code = [] 
pd_TotalVote = []



for district in list(list(list(root)[3])[0])[4]:   
    for party in list(district)[5]:
        for candidate in list(party)[2]:
            pd_DistrictName.append(list(district)[0].text)
            pd_DistrictNumber.append(int(list(district)[1].text))
            pd_DistrictQuota.append(float(list(district)[2].text))
            pd_DistrictMandates.append(list(district)[3].text)
            pd_partyName.append(list(party)[0].text)
            pd_partyCode.append(list(party)[1].text)
            pd_forename.append(list(candidate)[0].text)
            pd_surname.append(list(candidate)[1].text)
            pd_candidateRegNumber.append(int(list(candidate)[2].text))
            pd_candidateID.append(list(candidate)[3].text)
            for vote in list(candidate)[4]:
                vote_vector.append(list(vote)[1].text)
            #print(int(vote_vector[0]))
            pd_TotalVote.append(float(vote_vector[0]))
            vote_vector = []


    
    

raw_material = pd.DataFrame({'First Name':pd_forename,'Last Name':pd_surname,'RegNum':pd_candidateRegNumber,'Candidate ID':pd_candidateID,
                      'Total Votes':pd_TotalVote,'District Number':pd_DistrictNumber,'Quota':pd_DistrictQuota,'Mandates':pd_DistrictMandates,'Party Name':pd_partyName,
                      'Party Abbr.':pd_partyCode})  

raw_material['Candidates'] = raw_material['First Name'] +' '+ raw_material['Last Name']



def PartyNum(party):
    switcher={
        'RÜE':1,
        'EKRE':2,
        'REF':3,
        'IRL':4,
        'ROH':5,
        'EÜVP' : 6,
        'EVA':7 ,
        'KESK' : 8,
        'SDE': 9 ,
        'EIP' : 10,
        'Üksik.':11        
    }
    return switcher.get(party,"Invalid day of week")


raw_material['Party Number'] = list(map(PartyNum, raw_material['Party Abbr.']))



In [2]:
process = raw_material.copy()
clean = process[['Candidates','RegNum','Party Number','District Number','Mandates','Total Votes','Quota']]
clean

,Candidates,RegNum,Party Number,District Number,Mandates,Total Votes,Quota
0,VERONIKA ISBERG,101,1,1,9,55.0,6259.89
1,VADIM ŠAPTŠENKO,102,1,1,9,20.0,6259.89
2,AIMAR JUKKUM,103,1,1,9,14.0,6259.89
3,URMAS ESPENBERG,137,2,1,9,2087.0,6259.89
4,NELE VALGEPEA,138,2,1,9,281.0,6259.89
...,...,...,...,...,...,...,...
867,KADRI-AIJA VIIK,951,9,12,8,221.0,4932.38
868,PRIIT RUUT,948,9,12,8,191.0,4932.38
869,KALEV KALJUSTE,949,9,12,8,182.0,4932.38
870,ANDRES HIRVELA,950,9,12,8,144.0,4932.38


In [3]:
Candidates=np.array(clean['Candidates'].values.tolist())
RegNumbers=np.array(clean['RegNum'].values.tolist())
District=np.array(clean['District Number'].values.tolist())
Votes=np.array(clean['Total Votes'].values.tolist())
Mandate=np.array(clean['Mandates'].values.tolist())
PartyCode=np.array(clean['Party Number'].values.tolist())



def truncate(number, decimals=4):
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor


ElectoralThreshold=0.05*sum(Votes)



def District_Personal_Mandate_Distributation_by_Party(district):
    district_by_party_vector=[0]*12
    for i in range(len(Candidates)):
        distQuota = clean.loc[i]['Quota']
        if (Votes[i]>= distQuota and District[i]==district):
            district_by_party_vector[PartyCode[i]]=district_by_party_vector[PartyCode[i]]+1
    return district_by_party_vector



def Elligible_Parties():
    Elligibles=[]
    for i in range(11):
        vote = clean[clean['Party Number']== i]['Total Votes'].sum()
        if vote >=ElectoralThreshold:
            Elligibles.append(i)
    return Elligibles

In [4]:
personal = list(clean[clean['Total Votes'] >= clean['Quota']].index)

In [5]:

def District_Mandate_Party(i):
    vote = clean[clean['Party Number']== i]['Total Votes'].sum()
    if (i in Elligible_Parties()):
        print('This party has ', vote/ElectoralThreshold, 'mandates')
    else:
        print("This party is not elligible for regional mandates")
        
        
def Party_Per_District(party,district):
    sum=0
    if party in Elligible_Parties():
        for k in range(len(Candidates)):
            if PartyCode[k]== party and District[k]==district:
                sum = sum + Votes[k]
    else:
        sum = 0
    return sum




def District_Seat_Counter(party, district):
    distQuota = clean[clean['District Number'] == district]['Quota'].unique()[0]
    Seat_Numer=0
    if (party in Elligible_Parties()):
        Seat_Numer=math.floor(Party_Per_District(party, district) / distQuota )
        if (Party_Per_District(party, district) % distQuota >=  0.75 * distQuota ):
            Seat_Numer+=1
    else:
        Seat_Numer=0
    return  Seat_Numer





def District_Mandate_Vector(district):
    district_mandate=[0]*12
    for party in range(1,12):
        district_mandate[party]=District_Seat_Counter(party,district)-District_Personal_Mandate_Distributation_by_Party(district)[party]
    return district_mandate




def Allocating_District_Mandate( party , district) :
    distQuota = clean[clean['District Number'] == district]['Quota'].unique()[0]
    new = clean[(clean['Party Number'] == party) & (clean['District Number'] == district)]
    final_index=list(set(new.index).difference(set(personal)))
    df = clean.loc[final_index].sort_values(by=['Total Votes']).tail(District_Mandate_Vector(district)[party])
    dff = df[df['Total Votes'] >= 0.1 * distQuota]
    return (list(dff.index))



def District_Mandates_List():
    mandates_list=[]
    for party in tqdm(Elligible_Parties()):
        for district in tqdm(range(1,13)):
            mandates_list = mandates_list + Allocating_District_Mandate(party,district)
    return(mandates_list)




#####################################
distMandList = District_Mandates_List()
#####################################



def NationalPersonalParty():
    s = np.array([0]*12)
    for district in range(13):
        s = s + np.array(District_Personal_Mandate_Distributation_by_Party(district))
    return(s)








  0%|          | 0/6 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
districtElected = [273,
931,
862,
366,
262,
357,
769,
313,
902,
172,
806,
588,
422,
852,
602,
288,
827,
321,
921,
838,
786,
893,
777,
730,
462,
263,
446,
367,
761,
423,
412,
671,
912,
503,
300,
314,
934,
309,
274,
481,
753,
942,
347,
796,
387,
241,
878,
438,
808,
472,
628,
301,
776,
714,
726,
299,
492,
703,
729,
378,
398,
770,
740,
712,
328,
339]

In [10]:
set(list(clean.iloc[distMandList]['RegNum'].values)) == set( districtElected)

True

In [11]:
def District_National_By_Party(party):
    s = 0
    for i in range(1,13):
        s = s + len(Allocating_District_Mandate(party, i))
    return(s)

def TotalAlloctedForParty(party):
    return (NationalPersonalParty()[party]+District_National_By_Party(party))


def RemainedMandates():
    return (101 -(len(personal)+ len (distMandList)))



################




def Compensation_Elligible_Parties():
    CompensationElligibles=[]
    for i in range(12):
        vote = clean[clean['Party Number'] == i]['Total Votes'].sum()
        if vote >=ElectoralThreshold:
            CompensationElligibles.append(i)
    return CompensationElligibles




remMandate= RemainedMandates()
def ComparativeFigure(party):
    totalParty = TotalAlloctedForParty(party)
    if party in Compensation_Elligible_Parties():
        vector=[truncate(i ** 0.9,4) for i in range(totalParty +1,totalParty + remMandate+1)]
        s = np.array(vector)
        partyvote = clean[clean['Party Number'] == party]['Total Votes'].sum()
        vector=list(zip(list(map(truncate,list(partyvote /s))),[party] * remMandate))
    else :
        vector = None 
    return vector



def NationalComparativeFigures():
    nationalList=[]
    for party in tqdm(Compensation_Elligible_Parties()):
        nationalList = nationalList + ComparativeFigure(party)
    return nationalList
        


 



In [12]:
Rank = pd.read_excel(open('EstonianData.xlsx', 'rb'), sheet_name='Rank2015')

In [13]:
def RankMapping(reg): 
    if list(clean[clean['RegNum']==reg]['Party Number'])[0] == 11:
        return 0
    else:
        return list(Rank[(Rank['Name'] ==list(clean[clean['RegNum'] == reg]['Candidates'].values)[0]) & (Rank['PartyCode'] == list(clean[clean['RegNum']== reg]['Party Number'].values)[0])]['Rank'])[0]
    

clean['Rank'] = clean['RegNum']
clean['Rank'] = list(map(RankMapping,clean['RegNum']))    

/Users/ehsan.estaji/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ehsan.estaji/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [14]:
    
df = pd.DataFrame(NationalComparativeFigures())


removeCompensation = personal + distMandList
final_index=list(set(clean.index).difference(set(removeCompensation)))
clean_new =  clean.iloc[final_index]
dff = df.sort_values(by=0, ascending= False).head(remMandate)
CompensationSeatsPerParty = dff[1].value_counts()

100%|██████████| 6/6 [03:21<00:00, 33.64s/it]


In [15]:
def CompensationIndexSet():
    result=[]
    for party in Compensation_Elligible_Parties():
        compensResult = clean_new[(clean_new['Party Number'] == party) & (clean_new['Total Votes']>= 0.05* clean_new['Quota'])].sort_values(by='Rank', ascending = True).head(CompensationSeatsPerParty[party])
        result = result + list(compensResult.index)
    return result



In [16]:
clean.iloc[CompensationIndexSet()].index.size

22

In [17]:
compensElected = [ 
162,
243,
578,
212,
863,
672,
704,
290,
315,
778,
222,
922,
298,
715,
471,
590,
358,
903,
779,
349,
705,
491,
]

In [18]:
set(list(clean.iloc[CompensationIndexSet()]['RegNum'].values)) == set(compensElected)

True

In [19]:
raw_material.iloc[CompensationIndexSet()]

,First Name,Last Name,RegNum,Candidate ID,Total Votes,District Number,Quota,Mandates,Party Name,Party Abbr.,Candidates,Party Number
185,MARTIN,HELME,162,31722,4296.0,3,6526.25,8,Eesti Konservatiivne Rahvaerakond,EKRE,MARTIN HELME,2
735,ARNO,SILD,243,31810,523.0,11,4951.22,9,Eesti Konservatiivne Rahvaerakond,EKRE,ARNO SILD,2
535,JAAK,MADISON,212,31778,1883.0,8,5517.43,7,Eesti Konservatiivne Rahvaerakond,EKRE,JAAK MADISON,2
595,RAIVO,PÕLDARU,222,31793,919.0,9,5107.38,8,Eesti Konservatiivne Rahvaerakond,EKRE,RAIVO PÕLDARU,2
199,MARTIN,KUKK,290,31902,456.0,3,6526.25,8,Eesti Reformierakond,REF,MARTIN KUKK,3
381,LAURI,LUIK,315,31922,855.0,5,5612.17,6,Eesti Reformierakond,REF,LAURI LUIK,3
279,LAINE,RANDJÄRV,298,31915,1698.0,4,6269.93,14,Eesti Reformierakond,REF,LAINE RANDJÄRV,3
675,URMAS,KLAAS,358,31970,1501.0,10,5613.38,8,Eesti Reformierakond,REF,URMAS KLAAS,3
608,TERJE,TREI,349,31973,557.0,9,5107.38,8,Eesti Reformierakond,REF,TERJE TREI,3
615,JAAK,AAVIKSOO,471,32165,1405.0,9,5107.38,8,Erakond Isamaa ja Res Publica Liit,IRL,JAAK AAVIKSOO,4


In [20]:
clean[(clean['RegNum'] == 705) | (clean['RegNum']==795)]

,Candidates,RegNum,Party Number,District Number,Mandates,Total Votes,Quota,Rank
65,LAURI LAASI,705,8,1,9,407.0,6259.89,13
641,ALEKSANDR ŠIROKOV,795,8,9,8,1137.0,5107.38,85


In [21]:
set(list(clean.iloc[CompensationIndexSet()]['RegNum'].values)) == set(compensElected)

True